In [ ]:
# default_exp core

# Core utilities

> Utility functions and input folder

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export

import os

base_folder = os.environ.get(
    "S4REFSIMTOOL_INPUT", "/global/cfs/cdirs/cmbs4/dm/dstool/input/"
)

In [ ]:
# export

simulations_observing_efficiency = dict(
    pole=dict(SAT=0.3931149305555555, LAT=0.321544537037037),
    chile=dict(SAT=0.8656164351851852, LAT=0.917306712962963),
)

In [ ]:
#export


def get_telescope(channel):
    """Identify if a channel is in SAT or LAT

    Parameters
    ----------
    channel : str
        channel tag, e.g. MFH1

    Returns
    -------
    telescope : str
        'SAT' or 'LAT'
    """
    return channel[-2] + "AT"


def get_band(channel):
    """Identify the frequency band of a channel

    Currently it is just removing the last character

    Parameters
    ----------
    channel : str
        channel tag, e.g. MFH1

    Returns
    -------
    band : str
        'MFH'
    """
    return channel[:-1]


def get_observing_efficiency(observing_efficiency_config, site, telescope, band):
    """Extract the observing frequency for a specific channel from
    the configuration dictionary

    Parameters
    ----------
    observing_efficiency_config : dict
        the order of keys should be site, telescope, band
        e.g. {"Pole":{"LAT":{"HFL":0.1}}, "default":0.2}
    site : str
        site, Pole or Chile
    telescope : str
        LAT or SAT
    band: str
        e.g. HFL

    Returns
    -------
    observing_efficiency : float
        observing efficiency in decimal form, for example .2 for 20%
        """
    if site in observing_efficiency_config:
        if telescope in observing_efficiency_config[site]:
            try:
                return observing_efficiency_config[site][telescope][band]
            except KeyError:
                try:
                    return observing_efficiency_config[site][telescope]["default"]
                except KeyError:
                    pass
        try:
            return observing_efficiency_config[site]["default"]
        except KeyError:
            pass
    return observing_efficiency_config["default"]

In [ ]:
observing_efficiency_config =  {"Pole":{"LAT":{"HFL":0.1}}, "default":0.2}

In [ ]:
assert get_observing_efficiency(observing_efficiency_config, "Pole", "LAT", "HFL") == .1
assert get_observing_efficiency(observing_efficiency_config, "Pole", "LAT", "MFL") == .2
assert get_observing_efficiency(observing_efficiency_config, "Chile", "LAT", "MFL") == .2

In [ ]:
observing_efficiency_config_telescope_default =  {"Pole":{"LAT":{"HFL":0.1, "default":.15}}, "default":0.2}

assert get_observing_efficiency(observing_efficiency_config_telescope_default, "Pole", "LAT", "HFL") == .1
assert get_observing_efficiency(observing_efficiency_config_telescope_default, "Pole", "LAT", "MFL") == .15

In [ ]:
observing_efficiency_config_site_default =  {"Pole":{"default":.19, "LAT":{"HFL":0.1, "default":.15}}, "default":0.2}

assert get_observing_efficiency(observing_efficiency_config_site_default, "Pole", "LAT", "HFL") == .1
assert get_observing_efficiency(observing_efficiency_config_site_default, "Pole", "LAT", "MFL") == .15
assert get_observing_efficiency(observing_efficiency_config_site_default, "Pole", "SAT", "LFS") == .19

In [ ]:
# export

import h5py

s4 = h5py.File("cmbs4_tophat.h5", "r")


def parse_channels(channels):
    if channels in ["SAT", "LAT"]:
        channels = [ch for ch in s4.keys() if s4[ch].attrs["telescope"] == channels]
    elif channels in ["all", None]:
        channels = list(s4.keys())
    elif isinstance(channels, str):
        channels = channels.split(",")
    return channels

In [ ]:
assert parse_channels("LFL1") == ["LFL1"]
assert parse_channels(["LFL1"]) == ["LFL1"]
assert parse_channels("LFL1,LFL2") == ["LFL1", "LFL2"]
assert parse_channels("LAT") == ['HFL1', 'HFL2', 'LFL1', 'LFL2', 'MFL1', 'MFL2', 'ULFL1']
assert parse_channels("all") == list(s4.keys())